### CPT codes and descriptions (Medicare, 2012-2014)

Janos A. Perge, 09/26/2016

Purpose:   
1) Access data on the number and cost of medical procedures performed by Medicare providers.   
2) Obtain CPT procedure codes and their descriptions.
3) Repeat over 2012-2014 to get a more complete description list
    
-Get the Medicare-provider-charge data. This is a publicly available file on medical procedures and the associated cost performed by Medicare providers during year 2012-14. The data is downloadable from CMS:
https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Physician-and-Other-Supplier.html

Visit the url above, accept the CMS disclaimer, download and unzip file (2GB) and place it within the same directory as this script. The data is described in detail in 'Medicare-Physician-and-Other-Supplier-PUF-Methodology.PDF', included in this repository.  

#### Obtain R-packages

In [1]:
rm(list=ls())

packageList = c("data.table","stringr",'plyr')

is_installed <- function(mypkg) is.element(mypkg, installed.packages()[,1])

load_or_install<-function(package_names)
{
  for(package_name in package_names)
  {
    if(!is_installed(package_name))
    {
      install.packages(package_name,repos="http://lib.stat.cmu.edu/R/CRAN")
    }
    options(java.parameters = "-Xmx8g")
    library(package_name,character.only=TRUE,quietly=TRUE,verbose=FALSE)
  }
}

load_or_install(packageList)

#### Access data

In [21]:
cms_filename = "Medicare_Provider_Util_Payment_PUF_CY2014.txt" #data is also available on CMS for years 2013 and 2014
my_data_file = "procedures2014.RData"

start = Sys.time()
physician_data = data.frame(fread(cms_filename)) #This second way of reading data is ~5 times faster!
physician_data = physician_data[2:nrow(physician_data),]
colnames(physician_data) = tolower(colnames(physician_data))
    
Sys.time()-start

Read 9.8% of 9316308 rows

Warning message:
In fread(cms_filename): Bumped column 11 to type character on data row 1113710, field contains '96538-9100'. Coercing previously read values in this column from logical, integer or numeric back to character which may not be lossless; e.g., if '00' and '000' occurred before they will now be just '0', and there may be inconsistencies with treatment of ',,' and ',NA,' too (if they occurred in this column before the bump). If this matters please rerun and set 'colClasses' to 'character' for this column. Please note that column type detection uses the first 5 rows, the middle 5 rows and the last 5 rows, so hopefully this message should be very rare. If reporting to datatable-help, please rerun and include the output from verbose=TRUE.

Read 9316308 rows and 26 (of 26) columns from 1.945 GB file in 00:00:59


Time difference of 1.255336 mins

In [22]:
head(physician_data)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,average_medicare_standard_amt
2,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,99222,"Initial hospital inpatient care, typically 50 minutes per day",N,357,341,357,141.05,416.54621849,107.51787115,106.73123249
3,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,99223,"Initial hospital inpatient care, typically 70 minutes per day",N,98,98,98,207.79,611,152.55122449,153.30683673
4,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,99231,"Subsequent hospital inpatient care, typically 15 minutes per day",N,104,65,104,40.1,119,30.835384615,30.708269231
5,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,99232,"Subsequent hospital inpatient care, typically 25 minutes per day",N,1418,596,1418,73.66,217,56.297757405,56.098208745
6,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,99233,"Subsequent hospital inpatient care, typically 35 minutes per day",N,175,104,175,106.12,312,82.398171429,81.612742857
7,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,99238,"Hospital discharge day management, 30 minutes or less",N,330,316,330,74.14,217,55.681242424,55.199757576


In [23]:
colnames(physician_data)

[1] "npi"                              "nppes_provider_last_org_name"    
 [3] "nppes_provider_first_name"        "nppes_provider_mi"               
 [5] "nppes_credentials"                "nppes_provider_gender"           
 [7] "nppes_entity_code"                "nppes_provider_street1"          
 [9] "nppes_provider_street2"           "nppes_provider_city"             
[11] "nppes_provider_zip"               "nppes_provider_state"            
[13] "nppes_provider_country"           "provider_type"                   
[15] "medicare_participation_indicator" "place_of_service"                
[17] "hcpcs_code"                       "hcpcs_description"               
[19] "hcpcs_drug_indicator"             "line_srvc_cnt"                   
[21] "bene_unique_cnt"                  "bene_day_srvc_cnt"               
[23] "average_medicare_allowed_amt"     "average_submitted_chrg_amt"      
[25] "average_medicare_payment_amt"     "average_medicare_standard_amt"

In [24]:
#physician data is large and clogs memory. Therefore I take what I need and clear the rest from the workspace:
conversion_table = physician_data[, c('hcpcs_code', 'hcpcs_description')]
#rm(physician_data)

In [12]:
conversion_table12 = conversion_table[!duplicated(conversion_table$hcpcs_code),]
conversion_table12 = data.table(conversion_table12)
setkey(conversion_table12, hcpcs_code)
nrow(conversion_table12)
head(conversion_table12)

[1] 5949

,hcpcs_code,hcpcs_description
1,00100,Anesthesia for procedure on salivary gland with biopsy
2,00102,Anesthesia for procedure to repair lip defect present at birth
3,00103,Anesthesia for procedure on eyelid
4,00104,Anesthesia for electric shock treatment
5,00120,Anesthesia for biopsy of external middle and inner ear
6,00126,Anesthesia for incision of ear drum


In [19]:
conversion_table13 = conversion_table[!duplicated(conversion_table$hcpcs_code),]
conversion_table13 = data.table(conversion_table13)
setkey(conversion_table13, hcpcs_code)
nrow(conversion_table13)
head(conversion_table13)

[1] 5983

,hcpcs_code,hcpcs_description
1,00100,Anesthesia for procedure on salivary gland with biopsy
2,00103,Anesthesia for procedure on eyelid
3,00104,Anesthesia for electric shock treatment
4,00120,Anesthesia for biopsy of external middle and inner ear
5,00126,Anesthesia for incision of ear drum
6,00140,Anesthesia for procedure on eye


In [25]:
conversion_table14 = conversion_table[!duplicated(conversion_table$hcpcs_code),]
conversion_table14 = data.table(conversion_table14)
setkey(conversion_table14, hcpcs_code)
nrow(conversion_table14)
head(conversion_table14)

[1] 5972

,hcpcs_code,hcpcs_description
1,00100,Anesthesia for procedure on salivary gland with biopsy
2,00103,Anesthesia for procedure on eyelid
3,00104,Anesthesia for electric shock treatment
4,00120,Anesthesia for biopsy of external middle and inner ear
5,00126,Anesthesia for incision of ear drum
6,00140,Anesthesia for procedure on eye


In [28]:
# combine 3 years of data:
cTab <- rbind(conversion_table12,conversion_table13, conversion_table14)
cTab = unique(cTab)

In [29]:
nrow(cTab)

[1] 8118

In [30]:
head(cTab)

,hcpcs_code,hcpcs_description
1,00100,Anesthesia for procedure on salivary gland with biopsy
2,00102,Anesthesia for procedure to repair lip defect present at birth
3,00103,Anesthesia for procedure on eyelid
4,00104,Anesthesia for electric shock treatment
5,00120,Anesthesia for biopsy of external middle and inner ear
6,00126,Anesthesia for incision of ear drum


In [31]:
str(cTab)

Classes 'data.table' and 'data.frame':	8118 obs. of  2 variables:
 $ hcpcs_code       : chr  "00100" "00102" "00103" "00104" ...
 $ hcpcs_description: chr  "Anesthesia for procedure on salivary gland with biopsy" "Anesthesia for procedure to repair lip defect present at birth" "Anesthesia for procedure on eyelid" "Anesthesia for electric shock treatment" ...
 - attr(*, ".internal.selfref")=<externalptr> 


In [33]:
write.csv(cTab, file = "CPTCodesAndDesc2012_2014.txt", row.names=FALSE, na="")
save(cTab, file='CPTCodesAndDesc2012_2014.RData')

In [34]:
rm(physician_data)

In [35]:
conversion_table = cTab

#### Spreadsheet #2: HCPCS/CPT code to CCS conversion

In [36]:
ccs_file = '2016_ccs_services_procedures.csv'
ccs_table = data.table(read.csv(ccs_file))
ccs_table$Code.Range <- as.character(ccs_table$Code.Range)
setkey(ccs_table,Code.Range,CCS,CCS.Label)
head(ccs_table)

,Code.Range,CCS,CCS.Label
1,'0001T-0002T',52,"Aortic resection, replacement or anastomosis"
2,'0003T-0003T',130,"Other diagnostic procedures, female organs"
3,'0005T-0006T',59,Other OR procedures on vessels of head and neck
4,'0007T-0007T',211,Therapeutic radiology
5,'0008T-0008T',93,Other non-OR upper GI therapeutic procedures
6,'0009T-0009T',125,Other excision of cervix and uterus


In [37]:
# combine the above two tables

In [38]:
#create an incremental sequence of CSS codes from Code.Range:
get_code_range <- function(inp,ccscode,ccsdesc){
    
    code_range <- vector(mode="numeric", length=0) #empty 
    
    aaa = unlist(strsplit(inp, "-", fixed = TRUE))    
    aaa = sub("\'", "", aaa)
    
    if (!grepl("[a-zA-Z]", aaa[1])){  #if code does not contain letters
        aaa = as.numeric(aaa)
        code_range = seq.int(aaa[1],aaa[2])
        code_range = sprintf("%05d", code_range) # fixed width of five characters with leading zeros
        code_range = as.character(code_range)
        
    } else #if hcpcs code is alphanumeric, with the numeric part as an incremental sequence
    {
        bbb = substring(aaa[1], seq(1,nchar(aaa[1])), seq(1,nchar(aaa[1]),1)) #break up string to individual characters
        letterPos = grep("[a-zA-Z]", bbb, value = FALSE)
        letterChar = grep("[a-zA-Z]", bbb, value = TRUE)

        numericPart1 = grep("[0-9]", bbb, value = TRUE)
        numericPart1 = as.numeric(paste(numericPart1, collapse=""))

        bbb = substring(aaa[2], seq(1,nchar(aaa[2])), seq(1,nchar(aaa[2]),1)) 
        numericPart2 = grep("[0-9]", bbb, value = TRUE)
        numericPart2 = as.numeric(paste(numericPart2, collapse=""))

        cr  = seq.int(numericPart1,numericPart2)
        
        if (letterPos==1){
            code_range = sprintf("%s%04d", letterChar, cr) # fixed width of four characters with leading zeros
        } else
        {
            code_range = sprintf("%04d%s", cr, letterChar)
        }            
    }
    out = data.frame(hcpcs_code=code_range, ccs_code = ccscode, ccs_desc = ccsdesc)
}

In [39]:
# Expand CCS table, i.e. list every HCPCS code specified within the Code.Range
expanded_ccs = with(ccs_table, Map(get_code_range, Code.Range, CCS, CCS.Label))
expanded_ccs = rbind.fill(expanded_ccs)
expanded_ccs = data.table(expanded_ccs) 
setkey(expanded_ccs,hcpcs_code)

In [40]:
conversion_tab = merge(conversion_table,expanded_ccs, by='hcpcs_code', all.x=TRUE) #left join
conversion_table = merge(conversion_table,expanded_ccs, by='hcpcs_code') #inner join

In [41]:
head(conversion_tab)

,hcpcs_code,hcpcs_description,ccs_code,ccs_desc
1,00100,Anesthesia for procedure on salivary gland with biopsy,232,Anesthesia
2,00102,Anesthesia for procedure to repair lip defect present at birth,232,Anesthesia
3,00103,Anesthesia for procedure on eyelid,232,Anesthesia
4,00104,Anesthesia for electric shock treatment,232,Anesthesia
5,00120,Anesthesia for biopsy of external middle and inner ear,232,Anesthesia
6,00126,Anesthesia for incision of ear drum,232,Anesthesia


In [42]:
##These HCPCS codes don't have a match in the CCS table:
noMatch <- conversion_tab[is.na(ccs_code) | is.na(ccs_desc)]
noMatch

,hcpcs_code,hcpcs_description,ccs_code,ccs_desc
1,0015F,Melan follow-up complete,NA,NA
2,D7140,"Extraction, erupted tooth or exposed root (elevation and/or forceps removal)",NA,NA
3,D9940,"Occlusal guards, by report",NA,NA
4,G9001,"Coordinated care fee, initial rate",NA,NA
5,G9002,"Coordinated care fee, maintenance rate",NA,NA
6,G9003,"Coordinated care fee, risk adjusted high, initial",NA,NA
7,G9005,"Coordinated care fee, risk adjusted maintenance",NA,NA
8,G9008,"Coordinated care fee, physician coordinated care oversight services",NA,NA
9,G9009,"Coordinated care fee, risk adjusted maintenance, level 3",NA,NA
10,G9010,"Coordinated care fee, risk adjusted maintenance, level 4",NA,NA


In [43]:
#matching efficiency:
sum(!is.na(conversion_tab$ccs_code))/nrow(conversion_tab)

[1] 0.9966741

#### Save results

In [46]:
#csv files:
start = Sys.time()
# write.csv(conversion_table, file = "CPT_to_CCS_conversion.csv", row.names=FALSE, na="")

#Note that write.csv saves numeric hcpcs codes as an arabic number e.g. '102', omiting zero characters in the beginning.
#This would cause mismatches later when mapping fixed-length character arrays such as '00102'. 
#However, R's native data format saves the codes with leading zeros.

#R.data file:
save(conversion_tab, file='HCPCStoCCSConvTable2012_2014.RData')
write.csv(cTab, file = "HCPCStoCCSConvTable2012_2014.txt", row.names=FALSE, na="")

Sys.time()-start

Time difference of 0.09332418 secs

In [25]:
#rm(list=ls())